# Jeu de données 2 : Spam or ham?

In [ ]:
# 0 indique que ce n'est pas un spam
# 1 indique que c'est un spam

## Lecture du jeu de données et comptes

In [4]:
import pandas as pd
data_spam = pd.read_csv("spamham.csv")

In [5]:
texte = data_spam["text"]
print("Le texte comporte", len(texte), "mails.")

Le texte comporte 5728 mails.


In [4]:
import nltk
from nltk.tokenize import word_tokenize
# tokenizer comprenant les mots avec une apostrophe (l', qu') et les ponctuations séparément
# une telle tokenization nous permet d'avoir une bonne idée du nombre de token
nb_instances= 0
for i in texte:
    l=len(word_tokenize(i))
    nb_instances+=l
print("Il y a", nb_instances, " tokens en incluant la ponctuation.") # nombre de tokens avec ponctuation

import re
nb_instances2= 0
for i in texte:
    j = re.sub(",|;|\.", " ", i)
    l2=len(word_tokenize(j))
    nb_instances2 += l2
print("Il y a", nb_instances2, " tokens sans inclure la ponctuation.") # nombre de tokens sans ponctuation

from nltk.tokenize import sent_tokenize
nb_instances3= 0
for i in texte:
    longueur_text=len(sent_tokenize(i))
    nb_instances3 += longueur_text
print("Il y a", nb_instances3, "phrases dans l'ensemble du corpus.")

Il y a 1878684  tokens en incluant la ponctuation.
Il y a 1722545  tokens sans inclure la ponctuation.
Il y a 111600 phrases dans l'ensemble du corpus.


In [5]:
#1368 spams
ham = 5728-1368
print("Il y a 1368 spams au total et", ham, "hams")

Il y a 1368 spams au total et 4360 hams


## Vectorisation du jeu de données

In [6]:
mail = data_spam["text"]
print(mail)

0       Subject: naturally irresistible your corporate...
1       Subject: the stock trading gunslinger  fanny i...
2       Subject: unbelievable new homes made easy  im ...
3       Subject: 4 color printing special  request add...
4       Subject: do not have money , get software cds ...
                              ...                        
5723    Subject: re : research and development charges...
5724    Subject: re : receipts from visit  jim ,  than...
5725    Subject: re : enron case study update  wow ! a...
5726    Subject: re : interest  david ,  please , call...
5727    Subject: news : aurora 5 . 2 update  aurora ve...
Name: text, Length: 5728, dtype: object


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
## Récupérer les instances (X) et les classes (y) et vectoriser
y = data_spam["spam"]
V = CountVectorizer(ngram_range = (1,2) )
X = V.fit_transform(data_spam["text"])

## séparer train test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
test = 0.3*5728
train = 5728 - test

In [9]:
print("0,3 pour la taille du test soit environ", test, "mails. Donc environ", train, "mails pour le train.")

0,3 pour la taille du test soit environ 1718.3999999999999 mails. Donc environ 4009.6000000000004 mails pour le train.


## Classifications et Evaluations

### Le perceptron, réseau de neurones simple, classifieur linéaire

In [9]:
#classifier
from sklearn.linear_model import Perceptron

ppn = Perceptron(eta0=0.1, random_state=0)
ppn.fit(X_train, y_train)
y_pred = ppn.predict(X_test)

# On fait la somme de tous les cas où la valeur dans y_test est bien trouvée dans y_pred
print('Bons résultats %d' % (y_test == y_pred).sum())
print('Erreurs: %d' % (y_test != y_pred).sum())

Bons résultats 1685
Erreurs: 34


**Les données de classification : elles permettent d'évaluer la qualité de la classification. Ici, on calcule ces données avec les résultats donnés par le Perceptron.**

In [10]:
from sklearn.metrics import classification_report
#Classification report permet de msurer l'exactitude d'une clissification selon plusieurs paramètres

nom_classes = ["ham", "spam"]
report = classification_report(y_test, y_pred, target_names=nom_classes)
print(report)

              precision    recall  f1-score   support

         ham       0.99      0.98      0.99      1314
        spam       0.95      0.97      0.96       405

    accuracy                           0.98      1719
   macro avg       0.97      0.98      0.97      1719
weighted avg       0.98      0.98      0.98      1719



In [12]:
# la précision est la proportion des items pertinents parmi l'ensemble des items proposés 

In [13]:
# le rappel est la proportion des items pertinents proposés parmi l'ensemble des items pertinents. 

In [14]:
# Vrai négatif : absent, absent
# Vrai positif : présent, présent
# Faux négatif : présent(dans la référence), absent(dans l'hypothèse)
# Faux positif : absent, présent

In [15]:
# support : nombre d'instances concernées

# micro f-mesure : moyenne des F-mesure pondérée (une classe compte en fonction de sa taille)

# macro f-mesure : moyenne des F-mesure de chaque classe (indépendamment de leur taille)

In [15]:
#La matrice de confusion
from sklearn.metrics import confusion_matrix

matrice_confusion = confusion_matrix(y_test, y_pred)
print(matrice_confusion)

[[1256   58]
 [  33  372]]


In [17]:
# de gauche à droite et de bas en haut : 
# ham, spam
# spam, ham

### Deuxième évaluation : Un arbre de décision

In [13]:
from sklearn import tree
DT = tree.DecisionTreeClassifier()
DT = DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)

In [16]:
# encore une matrice de confusion
matrice_confusion = confusion_matrix(y_test, y_pred)
print(matrice_confusion)

print('Bons résultats %d' % (y_test == y_pred).sum())
print('Erreurs: %d' % (y_test != y_pred).sum())

[[1256   58]
 [  33  372]]
Bons résultats 1628
Erreurs: 91


In [17]:
from sklearn.metrics import classification_report
#Classification report permet de msurer l'exactitude d'une clissification selon plusieurs paramètres

nom_classes = ["ham", "spam"]
report = classification_report(y_test, y_pred, target_names=nom_classes)
print(report)

              precision    recall  f1-score   support

         ham       0.97      0.96      0.97      1314
        spam       0.87      0.92      0.89       405

    accuracy                           0.95      1719
   macro avg       0.92      0.94      0.93      1719
weighted avg       0.95      0.95      0.95      1719



In [57]:
# résultats moins bons que précédemment.

## Approfondissement

**Regardons l'impact du paramètres random_state**

In [18]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [19]:
print("Avec la valeur par défaut de random state")
for i in range(3):
  DT = tree.DecisionTreeClassifier()
  DT = DT.fit(X_train, y_train)
  y_pred = DT.predict(X_test)
  matrice_confusion = confusion_matrix(y_test, y_pred)
  print(matrice_confusion)
  stats = precision_recall_fscore_support(y_test, y_pred)
  print(stats)
  print('Bons résultats %d' % (y_test == y_pred).sum())
  print('Erreurs: %d' % (y_test != y_pred).sum())
  print("--"*10)

Avec la valeur par défaut de random state
[[1260   54]
 [  30  375]]
(array([0.97674419, 0.87412587]), array([0.95890411, 0.92592593]), array([0.96774194, 0.89928058]), array([1314,  405]))
Bons résultats 1635
Erreurs: 84
--------------------
[[1253   61]
 [  34  371]]
(array([0.97358197, 0.8587963 ]), array([0.95357686, 0.91604938]), array([0.96347559, 0.8864994 ]), array([1314,  405]))
Bons résultats 1624
Erreurs: 95
--------------------
[[1260   54]
 [  35  370]]
(array([0.97297297, 0.87264151]), array([0.95890411, 0.91358025]), array([0.96588731, 0.89264174]), array([1314,  405]))
Bons résultats 1630
Erreurs: 89
--------------------


In [21]:
from sklearn.metrics import accuracy_score
print ('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.95


In [22]:
print("En fixant random state")
for i in range(3):
  DT = tree.DecisionTreeClassifier(random_state=0)
  DT = DT.fit(X_train, y_train)
  y_pred = DT.predict(X_test)
  matrice_confusion = confusion_matrix(y_test, y_pred)
  print(matrice_confusion)
  stats = precision_recall_fscore_support(y_test, y_pred)
  print(stats)
  print('Bons résultats %d' % (y_test == y_pred).sum())
  print('Erreurs: %d' % (y_test != y_pred).sum())


En fixant random state
[[1261   53]
 [  37  368]]
(array([0.97149461, 0.87410926]), array([0.95966514, 0.90864198]), array([0.96554364, 0.89104116]), array([1314,  405]))
Bons résultats 1629
Erreurs: 90
[[1261   53]
 [  37  368]]
(array([0.97149461, 0.87410926]), array([0.95966514, 0.90864198]), array([0.96554364, 0.89104116]), array([1314,  405]))
Bons résultats 1629
Erreurs: 90
[[1261   53]
 [  37  368]]
(array([0.97149461, 0.87410926]), array([0.95966514, 0.90864198]), array([0.96554364, 0.89104116]), array([1314,  405]))
Bons résultats 1629
Erreurs: 90


In [24]:

print ('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.95


**Différents paramètres que l'on peut faire varier : max_depth, min_samples_split, min_samples_leaf et max_features**

In [91]:
print("On teste max_depth")
for i in range(1, 100):
  DT = tree.DecisionTreeClassifier(max_depth=i)
  DT = DT.fit(X_train, y_train)
  y_pred = DT.predict(X_test)
  matrice_confusion = confusion_matrix(y_test, y_pred)
  print("Avec max_depth=", i)
  print(matrice_confusion)
  stats = precision_recall_fscore_support(y_test, y_pred)
  print(stats)
  print('Bons résultats %d' % (y_test == y_pred).sum())
  print('Erreurs: %d' % (y_test != y_pred).sum())

On teste max_depth
Avec max_depth= 1
[[1314    0]
 [ 405    0]]
(array([0.76439791, 0.        ]), array([1., 0.]), array([0.86646884, 0.        ]), array([1314,  405]))
Bons résultats 1314
Erreurs: 405
Avec max_depth= 2
[[1036  278]
 [   1  404]]
(array([0.99903568, 0.59237537]), array([0.78843227, 0.99753086]), array([0.88132709, 0.74333027]), array([1314,  405]))
Bons résultats 1440
Erreurs: 279
Avec max_depth= 3
[[1089  225]
 [  11  394]]
(array([0.99     , 0.6365105]), array([0.82876712, 0.97283951]), array([0.90223695, 0.76953125]), array([1314,  405]))
Bons résultats 1483
Erreurs: 236
Avec max_depth= 4
[[1229   85]
 [ 132  273]]
(array([0.90301249, 0.76256983]), array([0.93531202, 0.67407407]), array([0.9188785 , 0.71559633]), array([1314,  405]))
Bons résultats 1502
Erreurs: 217
Avec max_depth= 5
[[1133  181]
 [   5  400]]
(array([0.99560633, 0.68846816]), array([0.86225266, 0.98765432]), array([0.92414356, 0.81135903]), array([1314,  405]))
Bons résultats 1533
Erreurs: 186
Avec

Avec max_depth= 43
[[1254   60]
 [  35  370]]
(array([0.97284717, 0.86046512]), array([0.9543379 , 0.91358025]), array([0.96350365, 0.88622754]), array([1314,  405]))
Bons résultats 1624
Erreurs: 95
Avec max_depth= 44
[[1252   62]
 [  31  374]]
(array([0.97583788, 0.85779817]), array([0.95281583, 0.92345679]), array([0.96418945, 0.88941736]), array([1314,  405]))
Bons résultats 1626
Erreurs: 93
Avec max_depth= 45
[[1258   56]
 [  29  376]]
(array([0.97746698, 0.87037037]), array([0.95738204, 0.92839506]), array([0.96732026, 0.89844683]), array([1314,  405]))
Bons résultats 1634
Erreurs: 85
Avec max_depth= 46
[[1250   64]
 [  32  373]]
(array([0.975039  , 0.85354691]), array([0.95129376, 0.92098765]), array([0.96302003, 0.88598575]), array([1314,  405]))
Bons résultats 1623
Erreurs: 96
Avec max_depth= 47
[[1262   52]
 [  34  371]]
(array([0.97376543, 0.87706856]), array([0.96042618, 0.91604938]), array([0.96704981, 0.89613527]), array([1314,  405]))
Bons résultats 1633
Erreurs: 86
Avec 

Avec max_depth= 85
[[1260   54]
 [  33  372]]
(array([0.97447796, 0.87323944]), array([0.95890411, 0.91851852]), array([0.96662831, 0.89530686]), array([1314,  405]))
Bons résultats 1632
Erreurs: 87
Avec max_depth= 86
[[1254   60]
 [  26  379]]
(array([0.9796875 , 0.86332574]), array([0.9543379 , 0.93580247]), array([0.96684657, 0.89810427]), array([1314,  405]))
Bons résultats 1633
Erreurs: 86
Avec max_depth= 87
[[1259   55]
 [  36  369]]
(array([0.97220077, 0.87028302]), array([0.95814307, 0.91111111]), array([0.96512074, 0.89022919]), array([1314,  405]))
Bons résultats 1628
Erreurs: 91
Avec max_depth= 88
[[1257   57]
 [  32  373]]
(array([0.97517455, 0.86744186]), array([0.956621  , 0.92098765]), array([0.96580868, 0.89341317]), array([1314,  405]))
Bons résultats 1630
Erreurs: 89
Avec max_depth= 89
[[1259   55]
 [  37  368]]
(array([0.97145062, 0.86997636]), array([0.95814307, 0.90864198]), array([0.96475096, 0.88888889]), array([1314,  405]))
Bons résultats 1627
Erreurs: 92
Avec 

Meilleur résultat obtenu :
Avec max_depth= 96
[[1265   49]
 [  30  375]]
(array([0.97683398, 0.88443396]), array([0.96270928, 0.92592593]), array([0.9697202 , 0.90470446]), array([1314,  405]))
Bons résultats 1640
Erreurs: 79

In [11]:
print("On teste min_samples_split:")
for i in range(1, 100):
  DT = tree.DecisionTreeClassifier(max_depth=i)
  DT = DT.fit(X_train, y_train)
  y_pred = DT.predict(X_test)
  matrice_confusion = confusion_matrix(y_test, y_pred)
  print("Avec max_depth=", i)
  print(matrice_confusion)
  stats = precision_recall_fscore_support(y_test, y_pred)
  print(stats)
  print('Bons résultats %d' % (y_test == y_pred).sum())
  print('Erreurs: %d' % (y_test != y_pred).sum())

On teste min_samples_split:
Avec max_depth= 1
[[1314    0]
 [ 405    0]]
(array([0.76439791, 0.        ]), array([1., 0.]), array([0.86646884, 0.        ]), array([1314,  405]))
Bons résultats 1314
Erreurs: 405


/home/alexane.jouglar/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Avec max_depth= 2
[[1036  278]
 [   1  404]]
(array([0.99903568, 0.59237537]), array([0.78843227, 0.99753086]), array([0.88132709, 0.74333027]), array([1314,  405]))
Bons résultats 1440
Erreurs: 279
Avec max_depth= 3
[[1089  225]
 [  11  394]]
(array([0.99     , 0.6365105]), array([0.82876712, 0.97283951]), array([0.90223695, 0.76953125]), array([1314,  405]))
Bons résultats 1483
Erreurs: 236
Avec max_depth= 4
[[1229   85]
 [ 132  273]]
(array([0.90301249, 0.76256983]), array([0.93531202, 0.67407407]), array([0.9188785 , 0.71559633]), array([1314,  405]))
Bons résultats 1502
Erreurs: 217
Avec max_depth= 5
[[1133  181]
 [   8  397]]
(array([0.99298861, 0.68685121]), array([0.86225266, 0.98024691]), array([0.92301426, 0.80773143]), array([1314,  405]))
Bons résultats 1530
Erreurs: 189
Avec max_depth= 6
[[1184  130]
 [  32  373]]
(array([0.97368421, 0.7415507 ]), array([0.90106545, 0.92098765]), array([0.93596838, 0.8215859 ]), array([1314,  405]))
Bons résultats 1557
Erreurs: 162
Avec ma

Avec max_depth= 44
[[1253   61]
 [  32  373]]
(array([0.97509728, 0.859447  ]), array([0.95357686, 0.92098765]), array([0.96421701, 0.88915375]), array([1314,  405]))
Bons résultats 1626
Erreurs: 93
Avec max_depth= 45
[[1258   56]
 [  30  375]]
(array([0.97670807, 0.87006961]), array([0.95738204, 0.92592593]), array([0.9669485 , 0.89712919]), array([1314,  405]))
Bons résultats 1633
Erreurs: 86
Avec max_depth= 46
[[1254   60]
 [  31  374]]
(array([0.97587549, 0.86175115]), array([0.9543379 , 0.92345679]), array([0.96498653, 0.89153754]), array([1314,  405]))
Bons résultats 1628
Erreurs: 91
Avec max_depth= 47
[[1259   55]
 [  33  372]]
(array([0.9744582 , 0.87119438]), array([0.95814307, 0.91851852]), array([0.96623177, 0.89423077]), array([1314,  405]))
Bons résultats 1631
Erreurs: 88
Avec max_depth= 48
[[1260   54]
 [  32  373]]
(array([0.9752322, 0.8735363]), array([0.95890411, 0.92098765]), array([0.96699923, 0.89663462]), array([1314,  405]))
Bons résultats 1633
Erreurs: 86
Avec ma

Avec max_depth= 86
[[1256   58]
 [  35  370]]
(array([0.97288923, 0.86448598]), array([0.95585997, 0.91358025]), array([0.96429942, 0.88835534]), array([1314,  405]))
Bons résultats 1626
Erreurs: 93
Avec max_depth= 87
[[1259   55]
 [  33  372]]
(array([0.9744582 , 0.87119438]), array([0.95814307, 0.91851852]), array([0.96623177, 0.89423077]), array([1314,  405]))
Bons résultats 1631
Erreurs: 88
Avec max_depth= 88
[[1254   60]
 [  33  372]]
(array([0.97435897, 0.86111111]), array([0.9543379 , 0.91851852]), array([0.96424452, 0.88888889]), array([1314,  405]))
Bons résultats 1626
Erreurs: 93
Avec max_depth= 89
[[1259   55]
 [  32  373]]
(array([0.97521301, 0.87149533]), array([0.95814307, 0.92098765]), array([0.96660269, 0.89555822]), array([1314,  405]))
Bons résultats 1632
Erreurs: 87
Avec max_depth= 90
[[1254   60]
 [  35  370]]
(array([0.97284717, 0.86046512]), array([0.9543379 , 0.91358025]), array([0.96350365, 0.88622754]), array([1314,  405]))
Bons résultats 1624
Erreurs: 95
Avec 

In [12]:
print("On teste min_samples_leaf:")
for i in range(1, 100):
  DT = tree.DecisionTreeClassifier(max_depth=i)
  DT = DT.fit(X_train, y_train)
  y_pred = DT.predict(X_test)
  matrice_confusion = confusion_matrix(y_test, y_pred)
  print("Avec max_depth=", i)
  print(matrice_confusion)
  stats = precision_recall_fscore_support(y_test, y_pred)
  print(stats)
  print('Bons résultats %d' % (y_test == y_pred).sum())
  print('Erreurs: %d' % (y_test != y_pred).sum())

On teste min_samples_leaf:
Avec max_depth= 1
[[1314    0]
 [ 405    0]]
(array([0.76439791, 0.        ]), array([1., 0.]), array([0.86646884, 0.        ]), array([1314,  405]))
Bons résultats 1314
Erreurs: 405


/home/alexane.jouglar/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Avec max_depth= 2
[[1036  278]
 [   1  404]]
(array([0.99903568, 0.59237537]), array([0.78843227, 0.99753086]), array([0.88132709, 0.74333027]), array([1314,  405]))
Bons résultats 1440
Erreurs: 279
Avec max_depth= 3
[[1089  225]
 [  11  394]]
(array([0.99     , 0.6365105]), array([0.82876712, 0.97283951]), array([0.90223695, 0.76953125]), array([1314,  405]))
Bons résultats 1483
Erreurs: 236
Avec max_depth= 4
[[1229   85]
 [ 132  273]]
(array([0.90301249, 0.76256983]), array([0.93531202, 0.67407407]), array([0.9188785 , 0.71559633]), array([1314,  405]))
Bons résultats 1502
Erreurs: 217
Avec max_depth= 5
[[1133  181]
 [   6  399]]
(array([0.99473222, 0.68793103]), array([0.86225266, 0.98518519]), array([0.92376682, 0.81015228]), array([1314,  405]))
Bons résultats 1532
Erreurs: 187
Avec max_depth= 6
[[1184  130]
 [  30  375]]
(array([0.9752883 , 0.74257426]), array([0.90106545, 0.92592593]), array([0.93670886, 0.82417582]), array([1314,  405]))
Bons résultats 1559
Erreurs: 160
Avec ma

Avec max_depth= 44
[[1248   66]
 [  29  376]]
(array([0.97729052, 0.85067873]), array([0.94977169, 0.92839506]), array([0.96333462, 0.88783943]), array([1314,  405]))
Bons résultats 1624
Erreurs: 95
Avec max_depth= 45
[[1247   67]
 [  33  372]]
(array([0.97421875, 0.84738041]), array([0.94901065, 0.91851852]), array([0.9614495 , 0.88151659]), array([1314,  405]))
Bons résultats 1619
Erreurs: 100
Avec max_depth= 46
[[1255   59]
 [  37  368]]
(array([0.97136223, 0.8618267 ]), array([0.95509893, 0.90864198]), array([0.96316193, 0.88461538]), array([1314,  405]))
Bons résultats 1623
Erreurs: 96
Avec max_depth= 47
[[1252   62]
 [  29  376]]
(array([0.97736144, 0.85844749]), array([0.95281583, 0.92839506]), array([0.96493256, 0.89205219]), array([1314,  405]))
Bons résultats 1628
Erreurs: 91
Avec max_depth= 48
[[1254   60]
 [  37  368]]
(array([0.97134005, 0.85981308]), array([0.9543379 , 0.90864198]), array([0.96276392, 0.88355342]), array([1314,  405]))
Bons résultats 1622
Erreurs: 97
Avec

Avec max_depth= 86
[[1259   55]
 [  36  369]]
(array([0.97220077, 0.87028302]), array([0.95814307, 0.91111111]), array([0.96512074, 0.89022919]), array([1314,  405]))
Bons résultats 1628
Erreurs: 91
Avec max_depth= 87
[[1259   55]
 [  27  378]]
(array([0.97900467, 0.87297921]), array([0.95814307, 0.93333333]), array([0.96846154, 0.90214797]), array([1314,  405]))
Bons résultats 1637
Erreurs: 82
Avec max_depth= 88
[[1252   62]
 [  31  374]]
(array([0.97583788, 0.85779817]), array([0.95281583, 0.92345679]), array([0.96418945, 0.88941736]), array([1314,  405]))
Bons résultats 1626
Erreurs: 93
Avec max_depth= 89
[[1256   58]
 [  31  374]]
(array([0.97591298, 0.86574074]), array([0.95585997, 0.92345679]), array([0.96578239, 0.89366786]), array([1314,  405]))
Bons résultats 1630
Erreurs: 89
Avec max_depth= 90
[[1251   63]
 [  28  377]]
(array([0.9781079 , 0.85681818]), array([0.95205479, 0.9308642 ]), array([0.96490551, 0.89230769]), array([1314,  405]))
Bons résultats 1628
Erreurs: 91
Avec 

In [13]:
print("On teste max_features:")
for i in range(1, 100):
  DT = tree.DecisionTreeClassifier(max_depth=i)
  DT = DT.fit(X_train, y_train)
  y_pred = DT.predict(X_test)
  matrice_confusion = confusion_matrix(y_test, y_pred)
  print("Avec max_depth=", i)
  print(matrice_confusion)
  stats = precision_recall_fscore_support(y_test, y_pred)
  print(stats)
  print('Bons résultats %d' % (y_test == y_pred).sum())
  print('Erreurs: %d' % (y_test != y_pred).sum())

On teste max_features:
Avec max_depth= 1
[[1314    0]
 [ 405    0]]
(array([0.76439791, 0.        ]), array([1., 0.]), array([0.86646884, 0.        ]), array([1314,  405]))
Bons résultats 1314
Erreurs: 405


/home/alexane.jouglar/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Avec max_depth= 2
[[1036  278]
 [   1  404]]
(array([0.99903568, 0.59237537]), array([0.78843227, 0.99753086]), array([0.88132709, 0.74333027]), array([1314,  405]))
Bons résultats 1440
Erreurs: 279
Avec max_depth= 3
[[1089  225]
 [  11  394]]
(array([0.99     , 0.6365105]), array([0.82876712, 0.97283951]), array([0.90223695, 0.76953125]), array([1314,  405]))
Bons résultats 1483
Erreurs: 236
Avec max_depth= 4
[[1229   85]
 [ 132  273]]
(array([0.90301249, 0.76256983]), array([0.93531202, 0.67407407]), array([0.9188785 , 0.71559633]), array([1314,  405]))
Bons résultats 1502
Erreurs: 217
Avec max_depth= 5
[[1133  181]
 [   5  400]]
(array([0.99560633, 0.68846816]), array([0.86225266, 0.98765432]), array([0.92414356, 0.81135903]), array([1314,  405]))
Bons résultats 1533
Erreurs: 186
Avec max_depth= 6
[[1184  130]
 [  34  371]]
(array([0.97208539, 0.74051896]), array([0.90106545, 0.91604938]), array([0.93522907, 0.81898455]), array([1314,  405]))
Bons résultats 1555
Erreurs: 164
Avec ma

Avec max_depth= 44
[[1258   56]
 [  37  368]]
(array([0.97142857, 0.86792453]), array([0.95738204, 0.90864198]), array([0.96435416, 0.88781665]), array([1314,  405]))
Bons résultats 1626
Erreurs: 93
Avec max_depth= 45
[[1260   54]
 [  37  368]]
(array([0.97147263, 0.87203791]), array([0.95890411, 0.90864198]), array([0.96514745, 0.88996372]), array([1314,  405]))
Bons résultats 1628
Erreurs: 91
Avec max_depth= 46
[[1256   58]
 [  34  371]]
(array([0.97364341, 0.86480186]), array([0.95585997, 0.91604938]), array([0.96466974, 0.88968825]), array([1314,  405]))
Bons résultats 1627
Erreurs: 92
Avec max_depth= 47
[[1257   57]
 [  38  367]]
(array([0.97065637, 0.86556604]), array([0.956621  , 0.90617284]), array([0.96358758, 0.8854041 ]), array([1314,  405]))
Bons résultats 1624
Erreurs: 95
Avec max_depth= 48
[[1261   53]
 [  32  373]]
(array([0.97525135, 0.87558685]), array([0.95966514, 0.92098765]), array([0.96739547, 0.8977136 ]), array([1314,  405]))
Bons résultats 1634
Erreurs: 85
Avec 

Avec max_depth= 86
[[1257   57]
 [  25  380]]
(array([0.98049922, 0.86956522]), array([0.956621 , 0.9382716]), array([0.96841294, 0.90261283]), array([1314,  405]))
Bons résultats 1637
Erreurs: 82
Avec max_depth= 87
[[1257   57]
 [  33  372]]
(array([0.9744186 , 0.86713287]), array([0.956621  , 0.91851852]), array([0.96543779, 0.89208633]), array([1314,  405]))
Bons résultats 1629
Erreurs: 90
Avec max_depth= 88
[[1252   62]
 [  33  372]]
(array([0.97431907, 0.85714286]), array([0.95281583, 0.91851852]), array([0.96344748, 0.88676996]), array([1314,  405]))
Bons résultats 1624
Erreurs: 95
Avec max_depth= 89
[[1256   58]
 [  31  374]]
(array([0.97591298, 0.86574074]), array([0.95585997, 0.92345679]), array([0.96578239, 0.89366786]), array([1314,  405]))
Bons résultats 1630
Erreurs: 89
Avec max_depth= 90
[[1259   55]
 [  29  376]]
(array([0.97748447, 0.87238979]), array([0.95814307, 0.92839506]), array([0.96771714, 0.89952153]), array([1314,  405]))
Bons résultats 1635
Erreurs: 84
Avec ma

In [ ]:
# pour ces trois derniers paramètres on ne passe jamais sous la barre des 85 erreurs.

### Autres caracétristiques

In [74]:
import statistics

X_stylo = []#notre nouvelle matrice de description
for text in data_spam["text"]:
    liste_mots = text.split()
    phrases = text.split(". ")
    NB_phrases = len(phrases)
    NB_mots = len(liste_mots)
    NB_caracteres = len(text)
    moyenne_taille_mots = statistics.mean([len(x) for x in liste_mots])
    moyenne_taille_phrases = NB_mots/NB_phrases
    caracteristiques = [NB_phrases, NB_mots, NB_caracteres, moyenne_taille_mots, moyenne_taille_phrases]
    X_stylo.append(caracteristiques)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X_stylo, y, test_size=0.3, random_state=0)
DT = tree.DecisionTreeClassifier()
DT = DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)
matrice_confusion = confusion_matrix(y_test, y_pred)
print(matrice_confusion)
stats = precision_recall_fscore_support(y_test, y_pred)
print(stats)
report = classification_report(y_test, y_pred, target_names=nom_classes)
print(report)

[[1103  211]
 [ 199  206]]
(array([0.84715822, 0.4940048 ]), array([0.83942161, 0.50864198]), array([0.84327217, 0.50121655]), array([1314,  405]))
              precision    recall  f1-score   support

         ham       0.85      0.84      0.84      1314
        spam       0.49      0.51      0.50       405

    accuracy                           0.76      1719
   macro avg       0.67      0.67      0.67      1719
weighted avg       0.76      0.76      0.76      1719



In [25]:
y_spam = [1 for x in y_test]##une liste remplie de 1
report = classification_report(y_test, y_spam, target_names=nom_classes)
matrice_confusion = confusion_matrix(y_test, y_spam)

y_ham = [0 for x in y_test]##une liste remplie de 0
report = classification_report(y_test, y_ham, target_names=nom_classes)
matrice_confusion = confusion_matrix(y_test, y_ham)
print(matrice_confusion)
print(report)

[[1314    0]
 [ 405    0]]
              precision    recall  f1-score   support

         ham       0.76      1.00      0.87      1314
        spam       0.00      0.00      0.00       405

    accuracy                           0.76      1719
   macro avg       0.38      0.50      0.43      1719
weighted avg       0.58      0.76      0.66      1719



/home/alexj/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Et maintenant on combine le BOW et le stylométrique**

In [77]:
## on regarde la "forme" de X
print(X.shape[0])#NB lignes   -> instances
print(X.shape[1])#Nb colonnes -> caractéristiques

##on crée une sparse matrix avec notre X_stylo
from scipy.sparse import csr_matrix
sparse_stylo = csr_matrix(X_stylo)
print(sparse_stylo.shape[0])#NB lignes   -> instances
print(sparse_stylo.shape[1])#Nb colonnes -> caractéristiques

## on a le même nombre de ligne, on fait donc une conctaténation horizontale :
from scipy.sparse import hstack
X_fusion = hstack((X, sparse_stylo))

5728
375535
5728
5


**Résultats:**

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X_fusion, y, test_size=0.3, random_state=0)
DT = tree.DecisionTreeClassifier()
DT = DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)
matrice_confusion = confusion_matrix(y_test, y_pred)
print(matrice_confusion)
stats = precision_recall_fscore_support(y_test, y_pred)
print(stats)
report = classification_report(y_test, y_pred, target_names=nom_classes)
print(report)
print('Bons résultats %d' % (y_test == y_pred).sum())
print('Erreurs: %d' % (y_test != y_pred).sum())

[[1258   56]
 [  36  369]]
(array([0.97217929, 0.86823529]), array([0.95738204, 0.91111111]), array([0.96472393, 0.88915663]), array([1314,  405]))
              precision    recall  f1-score   support

         ham       0.97      0.96      0.96      1314
        spam       0.87      0.91      0.89       405

    accuracy                           0.95      1719
   macro avg       0.92      0.93      0.93      1719
weighted avg       0.95      0.95      0.95      1719

Bons résultats 1627
Erreurs: 92


In [47]:
#résultats moyens

## Autres classifieurs

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings 
warnings.filterwarnings("ignore")

LR = LogisticRegression()
RDF = RandomForestClassifier()
SVC = SVC()
KN = KNeighborsClassifier()

Liste = [LR, RDF, SVC, KN]

In [46]:
for i in Liste:
    i = i.fit(X_train, y_train)
    y_pred = i.predict(X_test)
    matrice_confusion = confusion_matrix(y_test, y_pred)
    print(i)
    print(matrice_confusion)
    stats = precision_recall_fscore_support(y_test, y_pred)
    print(stats)
    report = classification_report(y_test, y_pred, target_names=nom_classes)
    print(report)
    print('Bons résultats %d' % (y_test == y_pred).sum())
    print('Erreurs: %d' % (y_test != y_pred).sum())
    print("-"*10)

LogisticRegression()
[[1306    8]
 [  14  391]]
(array([0.98939394, 0.97994987]), array([0.99391172, 0.9654321 ]), array([0.99164768, 0.97263682]), array([1314,  405]))
              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1314
        spam       0.98      0.97      0.97       405

    accuracy                           0.99      1719
   macro avg       0.98      0.98      0.98      1719
weighted avg       0.99      0.99      0.99      1719

Bons résultats 1697
Erreurs: 22
----------
RandomForestClassifier()
[[1313    1]
 [ 103  302]]
(array([0.92725989, 0.99669967]), array([0.99923896, 0.74567901]), array([0.96190476, 0.85310734]), array([1314,  405]))
              precision    recall  f1-score   support

         ham       0.93      1.00      0.96      1314
        spam       1.00      0.75      0.85       405

    accuracy                           0.94      1719
   macro avg       0.96      0.87      0.91      1719
weighted avg    

In [ ]:
# très bons résultats avec le LogisticRegression : ce sont les meilleurs obtenus jusqu'à maintenant

### En faisant varier les n-grammes

In [29]:
for min_N in range(1, 2):
  for max_N in range(min_N, 9):
    V = CountVectorizer(ngram_range = (min_N, max_N), analyzer = "char")
    X = V.fit_transform(data_spam["text"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for i in Liste:
        clf = i.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print('%s classifier : %.4f'%(i, score))

Ngram_range : (1, 1)
LogisticRegression() classifier : 0.8947
RandomForestClassifier() classifier : 0.9244
SVC() classifier : 0.7859
KNeighborsClassifier() classifier : 0.8662
Ngram_range : (1, 2)
LogisticRegression() classifier : 0.9581
RandomForestClassifier() classifier : 0.9471
SVC() classifier : 0.7882
KNeighborsClassifier() classifier : 0.8912
Ngram_range : (1, 3)
LogisticRegression() classifier : 0.9761
RandomForestClassifier() classifier : 0.9564
SVC() classifier : 0.8080
KNeighborsClassifier() classifier : 0.9133
Ngram_range : (1, 4)
LogisticRegression() classifier : 0.9820
RandomForestClassifier() classifier : 0.9651
SVC() classifier : 0.8173
KNeighborsClassifier() classifier : 0.9197
Ngram_range : (1, 5)
LogisticRegression() classifier : 0.9820
RandomForestClassifier() classifier : 0.9657
SVC() classifier : 0.8226
KNeighborsClassifier() classifier : 0.9267
Ngram_range : (1, 6)
LogisticRegression() classifier : 0.9843
RandomForestClassifier() classifier : 0.9604
SVC() classif

### Forêt aléatoire : zoom sur quelques paramètres

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
    # On fait la somme de tous les cas où la valeur dans y_test est bien trouvée dans y_pred
print('Bons résultats %d' % (y_test == y_pred).sum())
print('Erreurs: %d' % (y_test != y_pred).sum())

Bons résultats 1637
Erreurs: 82


In [49]:
# ci-dessus : paramètres par défaut

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

for i in range(1,100):
    clf = RandomForestClassifier(max_depth=i, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    # On fait la somme de tous les cas où la valeur dans y_test est bien trouvée dans y_pred
    print("Avec max_depth =", i)
    print('Bons résultats %d' % (y_test == y_pred).sum())
    print('Erreurs: %d' % (y_test != y_pred).sum())

Avec max_depth = 1
Bons résultats 1314
Erreurs: 405
Avec max_depth = 2
Bons résultats 1314
Erreurs: 405
Avec max_depth = 3
Bons résultats 1316
Erreurs: 403
Avec max_depth = 4
Bons résultats 1329
Erreurs: 390
Avec max_depth = 5
Bons résultats 1338
Erreurs: 381
Avec max_depth = 6
Bons résultats 1353
Erreurs: 366
Avec max_depth = 7
Bons résultats 1385
Erreurs: 334
Avec max_depth = 8
Bons résultats 1388
Erreurs: 331
Avec max_depth = 9
Bons résultats 1391
Erreurs: 328
Avec max_depth = 10
Bons résultats 1411
Erreurs: 308
Avec max_depth = 11
Bons résultats 1414
Erreurs: 305
Avec max_depth = 12
Bons résultats 1424
Erreurs: 295
Avec max_depth = 13
Bons résultats 1430
Erreurs: 289
Avec max_depth = 14
Bons résultats 1436
Erreurs: 283
Avec max_depth = 15
Bons résultats 1447
Erreurs: 272
Avec max_depth = 16
Bons résultats 1458
Erreurs: 261
Avec max_depth = 17
Bons résultats 1460
Erreurs: 259
Avec max_depth = 18
Bons résultats 1468
Erreurs: 251
Avec max_depth = 19
Bons résultats 1476
Erreurs: 243
Av

In [50]:
# amélioration des résultats jusqu'à max_depth = 95, puis augmentation du nombre d'erreurs
# mais moins bons résultats qu'avec les paramètres par défaut

### En supprimant la ponctuation et les stopwords

In [56]:
# rappels pour la cellule ci-après (ces éléments de codes ont été exécutés plus avant)
# LR = LogisticRegression()
# RDF = RandomForestClassifier()
# SVC = SVC()
# KN = KNeighborsClassifier()
# Liste = [LR, RDF, SVC, KN]

In [67]:
from nltk.corpus import stopwords

In [55]:
import re

In [68]:
def remove_ponctuation(chaine):
    ponctuations = [",", "'", '"', "-", "\."]
    for stopword in ponctuations:
        chaine = re.sub(f" {stopword} ", " ", chaine)
    return chaine

In [69]:
def remove_stopwords(chaine):
    final_stopwords_list = stopwords.words('english')
    s = chaine
    for stopword in final_stopwords_list:
        chaine = re.sub(f" {stopword} ", " ", chaine)
    return chaine

In [70]:
for pretraitement in ["stopwords", "ponctuation"]:
    if pretraitement =="stopwords":
        liste_pretraite = [remove_stopwords(j) for j in texte]
        X = V.fit_transform(liste_pretraite)
    elif pretraitement =="ponctuation":
        liste_titres_pretraite = [remove_ponctuation(j) for j in texte]
        X = V.fit_transform(liste_pretraite)    
    else:
        X = V.fit_transform(texte)
    for split_size in [0.3]: #[0.1, 0.2, 0.3, 0.9]:
        print(f"Split_size : {split_size}, Pretraitement: {pretraitement}")
    #découpage train VS test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_size, random_state=0)
    ##classification
        for i in Liste:
            i = i.fit(X_train, y_train)
            y_pred = i.predict(X_test)
            matrice_confusion = confusion_matrix(y_test, y_pred)
            print(i)
            print(matrice_confusion)
            stats = precision_recall_fscore_support(y_test, y_pred)
            print(stats)
            report = classification_report(y_test, y_pred, target_names=nom_classes)
            print(report)
            print('Bons résultats %d' % (y_test == y_pred).sum())
            print('Erreurs: %d' % (y_test != y_pred).sum())
            print ('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
            print("-"*10)

Split_size : 0.3, Pretraitement: stopwords
LogisticRegression()
[[1306    8]
 [  12  393]]
(array([0.9908953 , 0.98004988]), array([0.99391172, 0.97037037]), array([0.99240122, 0.9751861 ]), array([1314,  405]))
              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1314
        spam       0.98      0.97      0.98       405

    accuracy                           0.99      1719
   macro avg       0.99      0.98      0.98      1719
weighted avg       0.99      0.99      0.99      1719

Bons résultats 1699
Erreurs: 20
Accuracy: 0.99
----------
Accuracy: 0.99
RandomForestClassifier()
[[1308    6]
 [  36  369]]
(array([0.97321429, 0.984     ]), array([0.99543379, 0.91111111]), array([0.98419865, 0.94615385]), array([1314,  405]))
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1314
        spam       0.98      0.91      0.95       405

    accuracy                           0.98      17

In [30]:
# idem :

In [60]:
from sklearn.metrics import accuracy_score

liste_pretraite = [remove_ponctuation(j) for j in texte]
X = V.fit_transform(liste_pretraite)    
for split_size in [0.3]:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_size, random_state=0)

for i in Liste:
    i = i.fit(X_train, y_train)
    y_pred = i.predict(X_test)
    matrice_confusion = confusion_matrix(y_test, y_pred)
    print(i)
    print(matrice_confusion)
    stats = precision_recall_fscore_support(y_test, y_pred)
    print(stats)
    report = classification_report(y_test, y_pred, target_names=nom_classes)
    print(report)
    print('Bons résultats %d' % (y_test == y_pred).sum())
    print('Erreurs: %d' % (y_test != y_pred).sum())
    print ('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    print("-"*10)
#print(corpus_sans_stopwords)

LogisticRegression()
[[1305    9]
 [  11  394]]
(array([0.99164134, 0.97766749]), array([0.99315068, 0.97283951]), array([0.99239544, 0.97524752]), array([1314,  405]))
              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1314
        spam       0.98      0.97      0.98       405

    accuracy                           0.99      1719
   macro avg       0.98      0.98      0.98      1719
weighted avg       0.99      0.99      0.99      1719

Bons résultats 1699
Erreurs: 20
Accuracy: 0.99
----------
RandomForestClassifier()
[[1312    2]
 [  58  347]]
(array([0.95766423, 0.99426934]), array([0.99847793, 0.85679012]), array([0.97764531, 0.9204244 ]), array([1314,  405]))
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1314
        spam       0.99      0.86      0.92       405

    accuracy                           0.97      1719
   macro avg       0.98      0.93      0.95      1719
w

In [61]:
#LogisticRegression est le classifieur qui obtient à nouveau le meilleur résultat et 
#ils sont mêmes meilleurs que précédemment

In [ ]:
# accuracy commence à baisser à partir de split-size = 0.5
# faire varier eta0 ne change pas le résultat